In [1]:
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline 
import datetime
import calendar

In [2]:
def clean_data(df):
    df.Date = pd.to_datetime(df.Date)

    df['day_of_week'] = df['Date'].dt.weekday_name

    df = df.drop(['High','Low','Close','Volume'],axis=1)

    df['Consecutive trading day'] = df['day_of_week'].shift(1) + ' to ' + df['day_of_week']

    df['Overnight return'] = df['Open'] - df['Adj Close'].shift(1)

    df['Overnight % change'] = (df['Overnight return']/df['Adj Close'].shift(1))*100

    df['Overnight Volatility'] = df['Overnight % change'].rolling(window=20).std()

    df['Overnight Variance'] = df['Overnight Volatility'] ** 2
    
    df = df.dropna()

    return df

In [3]:
IXIC = pd.read_csv('/Users/Lee/Desktop/research-1/^IXIC-2.csv')
GSPC = pd.read_csv('/Users/Lee/Desktop/research-1/^GSPC-2.csv')

In [4]:
IXIC = clean_data(IXIC)
GSPC = clean_data(GSPC)

In [5]:
IXIC['Inday return'] = ((IXIC['Adj Close'] - IXIC['Open'])/IXIC['Open'])*100

## For IXIC:

In [6]:
conditional = [IXIC.groupby('day_of_week')]
mean = [cond.mean() for cond in conditional]
var = [cond.var() for cond in conditional]

In [7]:
mean

[                    Open    Adj Close  Overnight return  Overnight % change  \
 day_of_week                                                                   
 Friday       2837.175743  2836.235049          1.776173            0.046525   
 Monday       2832.420275  2830.604104          1.158587            0.052180   
 Thursday     2838.907390  2839.278197          1.429080            0.044599   
 Tuesday      2835.171240  2834.039780          2.281350            0.084066   
 Wednesday    2834.523230  2834.998560          1.339198            0.043472   
 
              Overnight Volatility  Overnight Variance  Inday return  
 day_of_week                                                          
 Friday                   0.676698            0.630570     -0.023829  
 Monday                   0.676517            0.628639     -0.070302  
 Thursday                 0.669991            0.621735      0.014640  
 Tuesday                  0.674133            0.625266     -0.018769  
 Wednesday  

**The table shows conditional mean of the overnight return and in day return (condition on the day of the week). **

**From the table, Tuesday has the highest mean of overnight return (2.28), Monday has the lowest mean of overnight return (1.15).**

** Also, Wednesday has the highest mean of in day return(0.04), Monday has the lowest mean of in day return (-0.07), which is consistent with the paper's conclusion.**

** Barttlet's test - test if different day's variance of the return is the same** 

In [8]:
scipy.stats.bartlett(IXIC.loc[IXIC['day_of_week']=='Monday']['Overnight % change'],
                    IXIC.loc[IXIC['day_of_week']=='Tuesday']['Overnight % change'],
                    IXIC.loc[IXIC['day_of_week']=='Wednesday']['Overnight % change'],
                    IXIC.loc[IXIC['day_of_week']=='Thursday']['Overnight % change'],
                    IXIC.loc[IXIC['day_of_week']=='Friday']['Overnight % change'])

BartlettResult(statistic=39.723593737728336, pvalue=4.93722513786907e-08)

** p = 4.9e-08 < 0.01, we reject null hypothesis that the variance of the return among different day of the week is the same, conclude the variance of the return is different (the volatility is different) ** 

## For old data of GSPC: Jan.1973 - Oct.1997, exclude 10 days before and after Oct.19 1987 

In [9]:
GSPC_OLD = pd.read_csv('/Users/Lee/Desktop/research-1/^GSPC-old.csv') 
# old data same as the paper 

In [10]:
GSPC_OLD = clean_data(GSPC_OLD)
GSPC_OLD_after = GSPC_OLD[GSPC_OLD['Date']>'1987-10-24']
GSPC_OLD_before = GSPC_OLD[GSPC_OLD['Date']<'1987-10-14']
GSPC_OLD = pd.concat([GSPC_OLD_before, GSPC_OLD_after])
GSPC_OLD['Inday return'] = ((GSPC_OLD['Adj Close'] - GSPC_OLD['Open'])/GSPC_OLD['Open'])*100

In [11]:
conditional_old = [GSPC_OLD.groupby('day_of_week')]
mean_old = [cond.mean() for cond in conditional_old]
var_old = [cond.var() for cond in conditional_old]

In [12]:
mean_old

[                   Open   Adj Close  Overnight return  Overnight % change  \
 day_of_week                                                                 
 Friday       264.731746  264.839984          0.002671            0.003598   
 Monday       264.504395  264.640101         -0.010664           -0.010478   
 Thursday     265.485560  265.529408          0.000904            0.001130   
 Tuesday      265.748075  265.986025          0.000699            0.001548   
 Wednesday    264.814544  265.021187         -0.002807           -0.002419   
 
              Overnight Volatility  Overnight Variance  Inday return  
 day_of_week                                                          
 Friday                   0.050005            0.010644      0.047872  
 Monday                   0.049528            0.010332     -0.003803  
 Thursday                 0.048359            0.010278      0.028271  
 Tuesday                  0.049232            0.010470      0.053933  
 Wednesday                

** From the table, the highest daily return is on Wednesday (0.084), the lowest daily return is on Monday(-0.0038), which agrees with the paper's result** 

In [13]:
scipy.stats.bartlett(GSPC_OLD.loc[GSPC_OLD['day_of_week']=='Monday']['Inday return'],
                    GSPC_OLD.loc[GSPC_OLD['day_of_week']=='Tuesday']['Inday return'],
                    GSPC_OLD.loc[GSPC_OLD['day_of_week']=='Wednesday']['Inday return'],
                    GSPC_OLD.loc[GSPC_OLD['day_of_week']=='Thursday']['Inday return'],
                    GSPC_OLD.loc[GSPC_OLD['day_of_week']=='Friday']['Inday return'])

BartlettResult(statistic=32.519662750404848, pvalue=1.4978954636015525e-06)

** Do the Bartlett test on the conditional mean,p = 1.5e-06 < 0.01, which means the mean of the return is significantly different on 1% level, condition on the day of the week. **

In [14]:
import arch

### GARCH(1,1): (standard GARCH model on in day return data)

In [15]:
am = arch.arch_model(GSPC_OLD['Inday return'])
res = am.fit(5)
GSPC_OLD['forecast_vol'] = np.sqrt(res.params['omega'] + res.params['alpha[1]'] * res.resid ** 2
                                  + res.params['beta[1]'] * res.conditional_volatility ** 2)
GSPC_OLD['Forecast_vol'] = GSPC_OLD['forecast_vol'].shift(1)
GSPC_OLD.drop(['forecast_vol'],axis=1,inplace=True)


Iteration:      5,   Func. Count:     40,   Neg. LLF: 7537.859245356119
Iteration:     10,   Func. Count:     74,   Neg. LLF: 7534.455477535987
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 7534.453692966564
            Iterations: 12
            Function evaluations: 86
            Gradient evaluations: 12


In [16]:
res.params

mu          0.043070
omega       0.004905
alpha[1]    0.036301
beta[1]     0.957480
Name: params, dtype: float64

** alpha[1] is VB, beta[1] is VA, omega is the constant VC, close to the result in the paper (slightly different because they used log return)**

### arch package in python doesn't support exogenous regressor in the conditional variance equation, cannot get the coefficients of the day of the week (dummy variables) 

In [17]:
a = pd.get_dummies(GSPC_OLD['day_of_week'])

In [18]:
am_2 = arch.arch_model(y=GSPC_OLD['Inday return'],x=a)

In [19]:
res_2 = am_2.fit(5)

Iteration:      5,   Func. Count:     40,   Neg. LLF: 7537.859245356119
Iteration:     10,   Func. Count:     74,   Neg. LLF: 7534.455477535987
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 7534.453692966564
            Iterations: 12
            Function evaluations: 86
            Gradient evaluations: 12


In [20]:
res_2.summary

<bound method ARCHModelResult.summary of                      Constant Mean - GARCH Model Results                      
Dep. Variable:           Inday return   R-squared:                      -0.000
Mean Model:             Constant Mean   Adj. R-squared:                 -0.000
Vol Model:                      GARCH   Log-Likelihood:               -7534.45
Distribution:                  Normal   AIC:                           15076.9
Method:            Maximum Likelihood   BIC:                           15103.9
                                        No. Observations:                 6228
Date:                Thu, Nov 02 2017   Df Residuals:                     6224
Time:                        18:01:46   Df Model:                            4
                                 Mean Model                                 
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
mu             0.